In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.mixed_precision import set_global_policy
from sklearn.model_selection import train_test_split

# Set mixed precision policy
set_global_policy('mixed_float16')

# Define transformer-based model
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate

        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def build(self, input_shape):
        super(TransformerBlock, self).build(input_shape)

    def get_config(self):
        config = super(TransformerBlock, self).get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'ff_dim': self.ff_dim,
            'rate': self.rate
        })
        return config

def create_transformer_model(input_shape, num_heads=2, ff_dim=16, dropout_rate=0.1):
    inputs = Input(shape=input_shape)
    transformer_block = TransformerBlock(input_shape[-1], num_heads, ff_dim, rate=dropout_rate)
    x = transformer_block(inputs)
    x = GlobalAveragePooling1D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(20, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Load metadata and scRNA-seq data
metadata_path = '/users/barmanjy/Desktop/1.GSE150949_metaData_with_lineage.txt'
data_path = '/users/barmanjy/Desktop/Persister Cell/GSE150949_scRNA.csv'

metadata_df = pd.read_csv(metadata_path, sep='\t')

# Function to load scRNA-seq data in chunks
def load_data_in_chunks(file_path, chunk_size=1000):
    data_chunks = []
    for chunk in pd.read_csv(file_path, engine='python', encoding='utf-8', chunksize=chunk_size):
        data_chunks.append(chunk)
    data = pd.concat(data_chunks, axis=0)
    return data

scRNA_data = load_data_in_chunks(data_path)

# Preprocess the data
def preprocess_data(metadata_df, scRNA_data):
    persister_metadata = metadata_df[metadata_df['sample_type'].str.endswith(('high', 'low'), na=False)]
    persister_metadata = persister_metadata.dropna(subset=['full_cell_barcode'])
    persister_metadata = persister_metadata[['full_cell_barcode', 'sample_name', 'sample_type']]
    persister_metadata['full_cell_barcode'] = persister_metadata['full_cell_barcode'].str.split('-').str[0]
    scRNA_data = scRNA_data.transpose()
    scRNA_data.reset_index(inplace=True)
    scRNA_data.rename(columns={'index': 'full_cell_barcode'}, inplace=True)
    scRNA_data['full_cell_barcode'] = scRNA_data['full_cell_barcode'].str.split('-').str[0]
    merged_data = pd.merge(scRNA_data, persister_metadata, on='full_cell_barcode', how='inner')
    X = merged_data.drop(columns=['full_cell_barcode', 'sample_name', 'sample_type'])
    y = merged_data['sample_type']
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    return X, y, merged_data, label_encoder

X, y, merged_data, label_encoder = preprocess_data(metadata_df, scRNA_data)

# Split the data into training (80%), validation (10%), and test (10%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Pad the training data and labels
def pad_data(X, y, batch_size):
    if X.shape[0] % batch_size != 0:
        padding_size = batch_size - (X.shape[0] % batch_size)
        X = np.pad(X, ((0, padding_size), (0, 0)), mode='constant')
        y = np.pad(y, (0, padding_size), mode='constant')
    return X, y

X_train, y_train = pad_data(np.array(X_train, dtype=np.float32), y_train, 8)
X_validation, y_validation = pad_data(np.array(X_validation, dtype=np.float32), y_validation, 8)
X_test, y_test = pad_data(np.array(X_test, dtype=np.float32), y_test, 8)

X_train = X_train[..., np.newaxis]
X_validation = X_validation[..., np.newaxis]
X_test = X_test[..., np.newaxis]

input_shape = (X_train.shape[1], 1)
model = create_transformer_model(input_shape)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_persister_model.keras', save_best_only=True, monitor='val_loss')

model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_validation, y_validation), callbacks=[early_stopping, model_checkpoint])

model.save('persister_model.keras')

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Generate classification report for the test set
y_test_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_test_pred.flatten(), target_names=label_encoder.classes_))

def load_trained_model(model_path='persister_model.keras'):
    return load_model(model_path, custom_objects={'TransformerBlock': TransformerBlock})

def predict_new_data(new_metadata_path, new_data_path, model, label_encoder, train_columns):
    new_metadata_df = pd.read_csv(new_metadata_path, sep='\t')
    new_scRNA_data = load_data_in_chunks(new_data_path)

    new_scRNA_data = new_scRNA_data.transpose()
    new_scRNA_data.reset_index(inplace=True)
    new_scRNA_data.rename(columns={'index': 'full_cell_barcode'}, inplace=True)
    new_scRNA_data['full_cell_barcode'] = new_scRNA_data['full_cell_barcode'].str.split('-').str[0]
    merged_new_data = pd.merge(new_scRNA_data, new_metadata_df, on='full_cell_barcode', how='inner')
    samples_to_predict = merged_new_data[merged_new_data['sample_name'].isin(['14_rep2_high', '14_rep2_low'])]
    X_new = samples_to_predict.drop(columns=['full_cell_barcode', 'sample_name', 'sample_type'])
    X_new = X_new.reindex(columns=train_columns, fill_value=0)
    X_new = np.array(X_new, dtype=np.float32)
    X_new = X_new[..., np.newaxis]
    
    # Create a tf.data.Dataset for prediction
    dataset = tf.data.Dataset.from_tensor_slices(X_new)
    dataset = dataset.batch(8)

    y_pred = (model.predict(dataset) > 0.5).astype("int32")
    y_pred_labels = label_encoder.inverse_transform(y_pred.flatten())
    return y_pred_labels

model = load_trained_model()
new_metadata_path = '/users/barmanjy/Desktop/1.GSE150949_metaData_with_lineage.txt'
new_data_path = '/users/barmanjy/Desktop/Persister Cell/GSE150949_scRNA.csv'
train_columns = list(X.columns)  # Get the column names from the original DataFrame
predictions = predict_new_data(new_metadata_path, new_data_path, model, label_encoder, train_columns)

# Compare predictions with sample_name '14_rep2_high'
comparison_sample = merged_data[merged_data['sample_name'] == '14_rep2_high']
X_comparison = comparison_sample.drop(columns=['full_cell_barcode', 'sample_name', 'sample_type'])
y_comparison = label_encoder.transform(comparison_sample['sample_type'])

X_comparison, y_comparison = pad_data(np.array(X_comparison, dtype=np.float32), y_comparison, 8)
X_comparison = X_comparison[..., np.newaxis]

y_pred_comparison = (model.predict(X_comparison) > 0.5).astype("int32")
print(classification_report(y_comparison, y_pred_comparison.flatten(), target_names=label_encoder.classes_))


KeyboardInterrupt: 

In [10]:
def preprocess_new_data(metadata_path, data_path, train_columns):
    new_metadata_df = pd.read_csv(metadata_path, sep='\t')
    new_scRNA_data = load_data_in_chunks(data_path).transpose()

    new_metadata_df['full_cell_barcode'] = new_metadata_df['full_cell_barcode'].str.split('-').str[0]
    new_scRNA_data.reset_index(inplace=True)
    new_scRNA_data.rename(columns={'index': 'full_cell_barcode'}, inplace=True)
    new_scRNA_data['full_cell_barcode'] = new_scRNA_data['full_cell_barcode'].str.split('-').str[0]

    merged_new_data = pd.merge(new_scRNA_data, new_metadata_df, on='full_cell_barcode', how='inner')
    print('After merge:')
    print(merged_new_data.head())

    # Adjust the filtering condition to capture 'high' or 'low' within 'sample_type'
    samples_to_predict = merged_new_data[merged_new_data['sample_type'].str.contains('high|low', case=False)]
    print('Samples to predict:')
    print(samples_to_predict.head())

    X_new = samples_to_predict.drop(columns=['full_cell_barcode', 'sample_name', 'sample_type'])
    X_new = X_new.reindex(columns=train_columns, fill_value=0)
    print('X_new after reindexing:')
    print(X_new.head())

    return np.array(X_new, dtype=np.float32)

def predict_new_data(metadata_path, data_path, model, label_encoder, train_columns):
    X_new = preprocess_new_data(metadata_path, data_path, train_columns)
    X_new = X_new[..., np.newaxis]
    print(f"New data shape after reindexing and reshaping: {X_new.shape}")

    # Create a tf.data.Dataset for prediction
    dataset = tf.data.Dataset.from_tensor_slices(X_new)
    dataset = dataset.batch(8)
    print('---Dataset')
    print(dataset)

    # Ensure dataset has at least one element by checking its length
    if len(list(dataset)) == 0:
        raise ValueError("The dataset is empty. Please ensure that the dataset contains elements.")

    # Perform prediction
    y_pred = (model.predict(dataset) > 0.5).astype("int32")
    y_pred_labels = label_encoder.inverse_transform(y_pred.flatten())
    return y_pred_labels

# Load the model and make predictions
model = load_trained_model()
predictions = predict_new_data(new_metadata_path, new_data_path, model, label_encoder, train_columns)

# Compare predictions with sample_type '14_high'
comparison_sample = merged_data[merged_data['sample_type'] == '14_high']
X_comparison = comparison_sample.drop(columns=['full_cell_barcode', 'sample_name', 'sample_type'])
y_comparison = label_encoder.transform(comparison_sample['sample_type'])

X_comparison, y_comparison = pad_data(np.array(X_comparison, dtype=np.float32), y_comparison, 8)
X_comparison = X_comparison[..., np.newaxis]

y_pred_comparison = (model.predict(X_comparison) > 0.5).astype("int32")
print(classification_report(y_comparison, y_pred_comparison.flatten(), target_names=label_encoder.classes_))



After merge:
  full_cell_barcode  RP11-34P13.7  FO538757.2  AP006222.2  RP4-669L17.2  \
0  TAAGCGTCAGCTTCGG           0.0    0.000000         0.0           0.0   
1  ATGCGATCAGCTTCGG           0.0    1.234842         0.0           0.0   
2  ATTACTCTCGGTCTAA           0.0    2.196514         0.0           0.0   
3  CGTCCATTCCCAAGTA           0.0    0.000000         0.0           0.0   
4  TGTCCCACAAGCGCTC           0.0    0.000000         0.0           0.0   

   RP4-669L17.10  RP5-857K21.4  RP5-857K21.2  RP11-206L10.9  FAM87B  ...  \
0            0.0           0.0           0.0            0.0     0.0  ...   
1            0.0           0.0           0.0            0.0     0.0  ...   
2            0.0           0.0           0.0            0.0     0.0  ...   
3            0.0           0.0           0.0            0.0     0.0  ...   
4            0.0           0.0           0.0            0.0     0.0  ...   

   nGene   nUMI  orig.ident  percent.mito  sample_id  time_point  \
0   2258   

2024-06-16 11:23:25.247981: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


2/2 ━━━━━━━━━━━━━━━━━━━━ 27s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 42s 42s/step
              precision    recall  f1-score   support

     14_high       1.00      1.00      1.00        16

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [ ]:
--code

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.mixed_precision import set_global_policy
from sklearn.model_selection import train_test_split

# Set mixed precision policy
set_global_policy('mixed_float16')

# Define transformer-based model
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate

        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def build(self, input_shape):
        super(TransformerBlock, self).build(input_shape)

    def get_config(self):
        config = super(TransformerBlock, self).get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'ff_dim': self.ff_dim,
            'rate': self.rate
        })
        return config

def create_transformer_model(input_shape, num_heads=2, ff_dim=16, dropout_rate=0.1):
    inputs = Input(shape=input_shape)
    transformer_block = TransformerBlock(input_shape[-1], num_heads, ff_dim, rate=dropout_rate)
    x = transformer_block(inputs)
    x = GlobalAveragePooling1D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(20, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Load metadata and scRNA-seq data
metadata_path = '/users/barmanjy/Desktop/1.GSE150949_metaData_with_lineage.txt'
data_path = '/users/barmanjy/Desktop/Persister Cell/GSE150949_scRNA.csv'

metadata_df = pd.read_csv(metadata_path, sep='\t')

# Function to load scRNA-seq data in chunks
def load_data_in_chunks(file_path, chunk_size=1000):
    data_chunks = []
    for chunk in pd.read_csv(file_path, engine='python', encoding='utf-8', chunksize=chunk_size):
        data_chunks.append(chunk)
    data = pd.concat(data_chunks, axis=0)
    return data

scRNA_data = load_data_in_chunks(data_path).transpose()

# Preprocess the data
def preprocess_data(metadata_df, scRNA_data):
    metadata_df['full_cell_barcode'] = metadata_df['full_cell_barcode'].str.split('-').str[0]
    scRNA_data.reset_index(inplace=True)
    scRNA_data.rename(columns={'index': 'full_cell_barcode'}, inplace=True)
    scRNA_data['full_cell_barcode'] = scRNA_data['full_cell_barcode'].str.split('-').str[0]
    
    persister_metadata = metadata_df[metadata_df['sample_type'].str.endswith(('high', 'low'), na=False)]
    persister_metadata = persister_metadata.dropna(subset=['full_cell_barcode'])
    persister_metadata = persister_metadata[['full_cell_barcode', 'sample_name', 'sample_type']]
    
    merged_data = pd.merge(scRNA_data, persister_metadata, on='full_cell_barcode', how='inner')
    X = merged_data.drop(columns=['full_cell_barcode', 'sample_name', 'sample_type'])
    y = merged_data['sample_type']
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    return X, y, merged_data, label_encoder

X, y, merged_data, label_encoder = preprocess_data(metadata_df, scRNA_data)

# Split the data into training (80%), validation (10%), and test (10%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Pad the training data and labels
def pad_data(X, y, batch_size):
    if X.shape[0] % batch_size != 0:
        padding_size = batch_size - (X.shape[0] % batch_size)
        X = np.pad(X, ((0, padding_size), (0, 0)), mode='constant')
        y = np.pad(y, (0, padding_size), mode='constant')
    return X, y

X_train, y_train = pad_data(np.array(X_train, dtype=np.float32), y_train, 8)
X_validation, y_validation = pad_data(np.array(X_validation, dtype=np.float32), y_validation, 8)
X_test, y_test = pad_data(np.array(X_test, dtype=np.float32), y_test, 8)

X_train = X_train[..., np.newaxis]
X_validation = X_validation[..., np.newaxis]
X_test = X_test[..., np.newaxis]

input_shape = (X_train.shape[1], 1)
model = create_transformer_model(input_shape)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_persister_model.keras', save_best_only=True, monitor='val_loss')

model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_validation, y_validation), callbacks=[early_stopping, model_checkpoint])

model.save('persister_model.keras')

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Generate classification report for the test set
y_test_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_test_pred.flatten(), target_names=label_encoder.classes_))

def load_trained_model(model_path='persister_model.keras'):
    return load_model(model_path, custom_objects={'TransformerBlock': TransformerBlock})

def preprocess_new_data(metadata_path, data_path, train_columns):
    new_metadata_df = pd.read_csv(metadata_path, sep='\t')
    new_scRNA_data = load_data_in_chunks(data_path).transpose()

    new_metadata_df['full_cell_barcode'] = new_metadata_df['full_cell_barcode'].str.split('-').str[0]
    new_scRNA_data.reset_index(inplace=True)
    new_scRNA_data.rename(columns={'index': 'full_cell_barcode'}, inplace=True)
    new_scRNA_data['full_cell_barcode'] = new_scRNA_data['full_cell_barcode'].str.split('-').str[0]

    merged_new_data = pd.merge(new_scRNA_data, new_metadata_df, on='full_cell_barcode', how='inner')
    print('After merge:')
    print(merged_new_data.head())

    # Adjust the filtering condition to capture 'high' or 'low' within 'sample_type'
    samples_to_predict = merged_new_data[merged_new_data['sample_type'].str.contains('high|low', case=False)]
    print('Samples to predict:')
    print(samples_to_predict.head())

    X_new = samples_to_predict.drop(columns=['full_cell_barcode', 'sample_name', 'sample_type'])
    X_new = X_new.reindex(columns=train_columns, fill_value=0)
    print('X_new after reindexing:')
    print(X_new.head())

    return np.array(X_new, dtype=np.float32)

def predict_new_data(metadata_path, data_path, model, label_encoder, train_columns):
    X_new = preprocess_new_data(metadata_path, data_path, train_columns)
    X_new = X_new[..., np.newaxis]
    print(f"New data shape after reindexing and reshaping: {X_new.shape}")

    # Create a tf.data.Dataset for prediction
    dataset = tf.data.Dataset.from_tensor_slices(X_new)
    dataset = dataset.batch(8)
    print('---Dataset')
    print(dataset)

    # Ensure dataset has at least one element by checking its length
    if len(list(dataset)) == 0:
        raise ValueError("The dataset is empty. Please ensure that the dataset contains elements.")

    # Perform prediction
    y_pred = (model.predict(dataset) > 0.5).astype("int32")
    y_pred_labels = label_encoder.inverse_transform(y_pred.flatten())
    return y_pred_labels

# Load the model and make predictions
model = load_trained_model()
predictions = predict_new_data(new_metadata_path, new_data_path, model, label_encoder, train_columns)

# Compare predictions with sample_type '14_high'
comparison_sample = merged_data[merged_data['sample_type'] == '14_high']
X_comparison = comparison_sample.drop(columns=['full_cell_barcode', 'sample_name', 'sample_type'])
y_comparison = label_encoder.transform(comparison_sample['sample_type'])

X_comparison, y_comparison = pad_data(np.array(X_comparison, dtype=np.float32), y_comparison, 8)
X_comparison = X_comparison[..., np.newaxis]

y_pred_comparison = (model.predict(X_comparison) > 0.5).astype("int32")
print(classification_report(y_comparison, y_pred_comparison.flatten(), target_names=label_encoder.classes_))


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 66s 66s/step - accuracy: 1.0000 - loss: 0.6931 - val_accuracy: 1.0000 - val_loss: 0.6927
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 61s 61s/step - accuracy: 1.0000 - loss: 0.6927 - val_accuracy: 1.0000 - val_loss: 0.6922
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 60s 60s/step - accuracy: 1.0000 - loss: 0.6922 - val_accuracy: 1.0000 - val_loss: 0.6917
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 62s 62s/step - accuracy: 1.0000 - loss: 0.6917 - val_accuracy: 1.0000 - val_loss: 0.6912
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 61s 61s/step - accuracy: 1.0000 - loss: 0.6912 - val_accuracy: 1.0000 - val_loss: 0.6907
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 60s 60s/step - accuracy: 1.0000 - loss: 0.6907 - val_accuracy: 1.0000 - val_loss: 0.6902
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 60s 60s/step - accuracy: 1.0000 - loss: 0.6902 - val_accuracy: 1.0000 - val_loss: 0.6897
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 60s 60s/step - accuracy: 1.0000 - loss: 0.6897 - val_accuracy: 1.0000 - val_loss: 0.6892


2024-06-16 11:58:40.422025: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/2 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/stepWARNING:tensorflow:6 out of the last 7 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7f6b9f483a30> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 27s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step
              precision    recall  f1-score   support

     14_high       1.00      1.00      1.00        16

    accuracy                           1.00        16
  